In [1]:
import pandas as pd

# Read CSV
df = pd.read_csv('./Dataset/Channel_Logs/channel2Logs.csv')

# Split in half without shuffling
mid_index = len(df) // 2
first_half = df.iloc[:mid_index]
second_half = df.iloc[mid_index:]

# Save to new CSV files
first_half.to_csv('./Dataset/Channel_Logs/channel2Logs.csv_part1.csv', index=False)
second_half.to_csv('./Dataset/Channel_Logs/channel2Logs.csv_part2.csv', index=False)


In [56]:
# Changes AWS to True if on SageMaker Instance and set S3 BUCKET and Key accordingly
AWS = False
REGION = 'eu-west-1'
BUCKET = 'ml-can-ids-logs'
s3 = None
from importlib import reload
import pandas as pd
import numpy as np
import os
import feature_selection
from feature_selection import feature_selection_preparation
from sklearn.ensemble import RandomForestClassifier

reload(feature_selection)

# Adjust pandas display optionss
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # No wrapping, long rows won't be cut off
pd.set_option('display.max_colwidth', None)  # Show full column content (especially useful for long strings)


# Remove this after testing/debugging
SEED = 42
np.random.seed(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'  



In [50]:
if AWS:
    s3 = get_s3_client(REGION, BUCKET, True)
    
    # Get S3 Object 
    channel2logs = s3.get_object(Bucket = BUCKET, Key= 'channel2Logs.csv')
    dos_attack_channel2 = s3.get_object(Bucket = BUCKET, Key= 'dos_attack_channel2.csv')
    replay_attack_channel2 = s3.get_object(Bucket = BUCKET, Key = 'replay_attack_channel2.csv') 
    spoofing_attack_channel2 = s3.get_object(Bucket = BUCKET, Key = 'new_spoofing_attack_channel2.csv') 

    channel2logs = channel2logs['Body'].read().decode('utf-8')
    dos_attack_channel2 = dos_attack_channel2['Body'].read().decode('utf-8')
    replay_attack_channel2 = replay_attack_channel2['Body'].read().decode('utf-8')
    spoofing_attack_channel2 = spoofing_attack_channel2['Body'].read().decode('utf-8')

    # Get Content
    channel2logs = StringIO(channel2logs)
    dos_attack_channel2 = StringIO(dos_attack_channel2)
    replay_attack_channel2 = StringIO(replay_attack_channel2)
    spoofing_attack_channel2 = StringIO(spoofing_attack_channel2)

    # Attack based on Channel
    preprocessed_DoS_channel2_csv_path = dos_attack_channel2 # DoS on channel 2 (Red Channel)
    preprocessed_Replay_channel2_csv_path = replay_attack_channel2 # Replay on channel 2 (Red Channel)
    preprocessed_Spoofing_channel2_csv_path = spoofing_attack_channel2 # Spoofing on channel 2 (Red Channel)

    # Unprocessed Channel Data
    preprocessed_normal_channel2_csv_path = channel2logs # Red Channel
    preprocessed_normal_channel4_csv_path = "" # Yellow Channel
    preprocessed_normal_channel5_csv_path = "" # Green Channel

    # Current best model
    best_model_path = ""
else:
    # Unprocessed Normal and Attack Data
    preprocessed_normal_csv_path = './Dataset/Tw22206_L003_with_ecu_channel.csv'  # Normal Unprocessed
    preprocessed_DoS_csv_path = './Dataset/Attack_Logs/dos_attack.csv'  # Dos Unprocessed
    preprocessed_Fuzzy_csv_path = './Dataset/Attack_Logs/fuzzy_attack.csv'  # Fuzzy Unprocessed
    preprocessed_Replay_csv_path = './Dataset/Attack_Logs/replay_attack.csv'  # Replay Unprocessed - Test
    preprocessed_Spoofing_csv_path = './Dataset/Attack_Logs/spoofing_attack.csv'  # Spoofing Unprocessed
    preprocessed_Suspension_csv_path = './Dataset/Attack_Logs/suspension_attack.csv'  # Suspension Unprocessed - Hardest Attack Type


    # Attack based on Channel
    preprocessed_DoS_channel2_csv_path = './Dataset/Attack_Logs/dos_attack_channel2.csv'  # DoS on channel 2 (Red Channel)
    preprocessed_Replay_channel2_csv_path = './Dataset/Attack_Logs/replay_attack_channel2.csv'  # Replay on channel 2 (Red Channel)
    preprocessed_Suspension_channel2_csv_path = './Dataset/Attack_Logs/suspension_attack_channel2.csv'  # Suspension on channel 2 (Red Channel)
    preprocessed_Spoofing_channel2_csv_path = './Dataset/Attack_Logs/spoofing_attack_channel2.csv'  # Spoofing on channel 2 (Red Channel)
    preprocessed_new_Spoofing_channel2_csv_path = './Dataset/Attack_Logs/new_spoofing_attack_channel2.csv'  # Spoofing on channel 2 (Red Channel)
    preprocessed_parkingBrake_Spoofing_channel2_csv_path = './Dataset/Attack_Logs/ParkingBrakeController_EPB__spoofing_attack_channel2.csv'  # Spoofing on channel 2 (Red Channel)

    # Unprocessed Channel Data
    preprocessed_normal_channel0_csv_path = './Dataset/Channel_Logs/channel0Logs.csv'
    preprocessed_normal_channel2_csv_path = './Dataset/Channel_Logs/channel2Logs.csv'  # Red Channel
    preprocessed_normal_channel4_csv_path = './Dataset/Channel_Logs/channel4Logs.csv'  # Yellow Channel
    preprocessed_normal_channel5_csv_path = './Dataset/Channel_Logs/channel5Logs.csv'  # Green Channel


    # Preprocessed Dataframe Data
    processeddataframe_normal_csv_path = './Dataset/Processed_Dataframes/train_dataframe.csv'  # Normal CSV Dataframe (Turns Lists into Strings)
    processeddataframe_DoS_csv_path = './Dataset/Processed_Dataframes/test_DoS_dataframe.csv'  # DoS CSV Dataframe (Turns Lists into Strings)

    # Preprocessed Pickle Data
    processeddataframe_normal_pickle_path = './Dataset/Processed_Dataframes/train_Normal_dataframePickle.pkl'  # Normal Pickle Dataframe
    processeddataframe_DoS_pickle_path = './Dataset/Processed_Dataframes/test_DoS_dataframePickle.pkl'  # DoS Pickle Dataframe

    # Current best model
    best_model_path = "./Resources/Models/SOA_VAE_E6_LD38_EP30_NT100000_B1024_I42.keras"



    # PRELOAD Dataframe for Debug
    DEBUG = False 


In [ ]:
LOAD_DATAFRAME = False
TS_FRESH = False

n_rows_train = 40000    # select how many rows to load. None if whole train datasset
n_rows_test = 40000   # select how many rows to load. None if whole test datasset
batch_size = 1024
window_size = 50    # increase window size
stride = 1         # increase stride as a buffer
split_ratio = 0.8     # % of training data to use for training
window_anomaly_ratio = 0.1   # For 1 anomaly per window do: 1 / window_size

# Preprocess and load training data
processeddataframe = feature_selection_preparation(preprocessed_normal_channel2_csv_path, 'training', rows=n_rows_train)

#processeddataframe_test = feature_selection_preparation(preprocessed_new_Spoofing_channel2_csv_path, 'test', rows=n_rows_test, ts_fresh= TS_FRESH, ts_fresh_parameters= tsfresh_features, ts_fresh_custom_features= custom_fc_parameters)
processeddataframe_test = feature_selection_preparation(preprocessed_new_Spoofing_channel2_csv_path, 'test', rows=n_rows_test)



#############START#####################
Average time for ID calculations: 0.000043 seconds
Average time for payload AVERAGE calculations: 0.000014 seconds
Average time for payload CHANGE calculations: 0.000023 seconds
Average time for Entropy per ID calculations: 0.000019 seconds
Compute Temporal Features completed 4.05 seconds
Feature Selection completed in 10.27 seconds
#############START#####################
Raw type values: ['R' 'T']
Normal entries in 'type' column : 35333
Anomalies in 'type' column: 4667
Average time for ID calculations: 0.000043 seconds
Average time for payload AVERAGE calculations: 0.000010 seconds
Average time for payload CHANGE calculations: 0.000021 seconds
Average time for Entropy per ID calculations: 0.000020 seconds
Compute Temporal Features completed 3.83 seconds
Feature Selection completed in 9.77 seconds


In [55]:
print(processeddataframe.head(3))
print("------------------------------")

print(processeddataframe['features'].head(1))
print("------------------------------")
print(processeddataframe_test['features'].head(1))
print("------------------------------")
#only_attack_df = processeddataframe_test[processeddataframe_test['type'] == 1]
#only_attack_df = only_attack_df['features']
#print(only_attack_df.head(2))

   bit_0  bit_1  bit_2  bit_3  bit_4  bit_5  bit_6  bit_7  bit_8  bit_9  \
0      0      1      1      0      0      1      1      1      1      1   
1      0      1      1      0      0      1      1      1      1      1   
2      0      1      1      0      0      1      1      1      1      1   

   bit_10  bit_11  bit_12  bit_13  bit_14  bit_15  bit_16  bit_17  bit_18  \
0       1       1       1       0       1       1       1       0       1   
1       1       1       1       0       1       1       0       0       0   
2       1       1       1       1       1       1       0       1       1   

   bit_19  bit_20  bit_21  bit_22  bit_23  bit_24  bit_25  bit_26  bit_27  \
0       0       0       0       0       1       0       0       1       1   
1       1       1       0       0       1       0       0       1       1   
2       1       0       0       0       1       0       0       1       1   

   bit_28  timestamp  arbitration_id  channel  dlc                     data  \
0 

**Mean and STD Between Normal and Attack Dataset**

In [32]:

# Convert 'features' column to DataFrame
normal_features_df = pd.DataFrame(processeddataframe['features'].tolist())
attack_features_df = pd.DataFrame(processeddataframe_test['features'].tolist())

# Compute mean difference
mean_diff = normal_features_df.mean() - attack_features_df.mean()

# Compute standard deviation difference
std_diff = normal_features_df.std() - attack_features_df.std()

# Create a DataFrame with mean and std differences
diff_df = pd.DataFrame({
    'Mean Difference': mean_diff,
    'Std Difference': std_diff
})

# Compute absolute mean difference and sort by it
diff_df['Abs Mean Difference'] = diff_df['Mean Difference'].abs()
diff_df = diff_df.sort_values(by='Abs Mean Difference', ascending=False)

# Print results
print(diff_df)
print("\nFeature Index Mapping:")
print("0-28:  Arbitration ID")
print("29-36: Payload")
print("37:  Message Frequency")
print("38:  Timestamp")
print("39:  Payload Entropy")
print("40: Entropy Average over 2 ms")
print("41: Entropy per ID")
print("42: Entropy per ID STD")

    Mean Difference  Std Difference  Abs Mean Difference
37         0.145030        0.021424             0.145030
40        -0.140575        0.090427             0.140575
23        -0.078950       -0.017097             0.078950
26        -0.067550       -0.002347             0.067550
42        -0.065846       -0.156513             0.065846
25         0.044925        0.014467             0.044925
22         0.043650        0.014972             0.043650
28        -0.042925        0.015410             0.042925
16        -0.037825       -0.003627             0.037825
41        -0.037646        0.016759             0.037646
39        -0.037474       -0.003801             0.037474
13         0.036450        0.008849             0.036450
18        -0.036175        0.001240             0.036175
17        -0.035675        0.008937             0.035675
10         0.028625       -0.011310             0.028625
24         0.028025        0.019529             0.028025
14         0.024775        0.00

In [61]:
# Convert 'features' column to DataFrame
normal_features_df = pd.DataFrame(processeddataframe['features'].tolist())
attack_features_df = pd.DataFrame(processeddataframe_test['features'].tolist())

#attack_features_df = pd.DataFrame(only_attack_df.tolist())

# Define window size
window_size = 200
num_windows = min(len(normal_features_df) // window_size, len(attack_features_df) // window_size)

# Store results for each window
window_results = []

for i in range(num_windows):
    # Select windowed data
    normal_window = normal_features_df.iloc[i * window_size: (i + 1) * window_size]
    attack_window = attack_features_df.iloc[i * window_size: (i + 1) * window_size]

    # Compute mean difference for the window
    mean_diff = normal_window.mean() - attack_window.mean()

    # Store results
    window_results.append(mean_diff)

# Convert results to DataFrame
window_diff_df = pd.DataFrame(window_results)

# Compute overall statistics (mean and standard deviation of differences across windows)
final_diff_df = pd.DataFrame({
    'Mean Difference': window_diff_df.mean(),
    'Std of Difference': window_diff_df.std()
})

# Compute absolute mean difference and sort
final_diff_df['Abs Mean Difference'] = final_diff_df['Mean Difference'].abs()
final_diff_df = final_diff_df.sort_values(by='Abs Mean Difference', ascending=False)

# Print results
print(final_diff_df)
print("\nFeature Index Mapping:")
print("0-28:  Arbitration ID")
print("29-36: Payload")
print("37:  Message Frequency")
print("38:  Timestamp")
print("39:  Payload Entropy")
print("40: Entropy Average over 2 ms")
print("41: Entropy per ID")
print("42: Entropy per ID STD")


    Mean Difference  Std of Difference  Abs Mean Difference
37         0.145030           0.057814             0.145030
40        -0.140575           0.146190             0.140575
23        -0.078950           0.027866             0.078950
26        -0.067550           0.025696             0.067550
42        -0.065846           0.023277             0.065846
25         0.044925           0.021967             0.044925
22         0.043650           0.018443             0.043650
28        -0.042925           0.016762             0.042925
16        -0.037825           0.016278             0.037825
41        -0.037646           0.024716             0.037646
39        -0.037474           0.015354             0.037474
13         0.036450           0.020931             0.036450
18        -0.036175           0.015969             0.036175
17        -0.035675           0.021027             0.035675
10         0.028625           0.014750             0.028625
24         0.028025           0.019087  

In [45]:
def binary_encode_integers(ids, num_bits):
    """
    Converts a list of CAN IDs (integers) to binary representation.

    Args:
        ids (list of int): List of CAN IDs as integers (e.g., [452948266, 452946218]).
        num_bits (int): Number of bits to represent the IDs in binary format.

    Returns:
        numpy.ndarray: Array of binary representations, where each row is a binary vector.
    """
    binary_ids = []
    for id_int in ids:
        # Convert integer to binary and pad with leading zeros
        binary_vector = [int(bit) for bit in f"{id_int:0{num_bits}b}"]
        binary_ids.append(binary_vector)
    return np.array(binary_ids)

In [49]:
binary = [218068007, 218063655, 218099239]

print("Original Integers", binary)
print("Binary" , binary_encode_integers(binary, 29))

Original Integers [218068007, 218063655, 218099239]
Binary [[0 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 0 0 0 1 0 0 1 1 1]
 [0 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 0 0 1 0 0 1 1 1]
 [0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 1 0 0 1 1 1]]


In [58]:
binary = [218068007, 218063655, 218099239]

print("Original Integers", binary)
print("Binary" , binary_encode_integers(binary, 29))

Original Integers [218068007, 218063655, 218099239]
Binary [[0 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 0 0 0 1 0 0 1 1 1]
 [0 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 0 0 1 0 0 1 1 1]
 [0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 1 0 0 1 1 1]]


In [ ]:
import pandas as pd
import re

# Prepare lists to store parsed data
timestamps = []
ids = []
dlcs = []
datas = []

# Open and read the file
with open('./Dataset/HCRL_CarHacking/Normal_dataset.txt', 'r') as f:
    for line in f:
        # Use regular expressions to extract fields
        match = re.search(r'Timestamp:\s+([0-9.]+)\s+ID:\s+([0-9a-fA-F]+).*DLC:\s+(\d+)\s+(.+)', line)
        if match:
            timestamp = float(match.group(1))
            arbitration_id = int(match.group(2), 16)  # convert hex to int
            dlc = int(match.group(3))
            data = match.group(4).strip().lower()
            # Store results
            timestamps.append(timestamp)
            ids.append(arbitration_id)
            dlcs.append(dlc)
            datas.append(data)

# Create DataFrame
df = pd.DataFrame({
    'timestamp': timestamps,
    'arbitration_id': ids,
    'dlc': dlcs,
    'data': datas
})

# Optionally specify dtypes
dtypes_train = {
    'timestamp': float,
    'arbitration_id': int,
    'dlc': int,
    'data': str,
}
df = df.astype(dtypes_train)

# Save to Excel
df.to_csv('./Dataset/HCRL_CarHacking/Normal_dataset.csv', index=False)

OSError: Cannot save file into a non-existent directory: 'HCRL_CarHacking'